In [ ]:
from IPython.display import HTML, display
import torch
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import pandas as pd
import json

In [ ]:
def find_all(a_str, sub):
    start = 0
    res = []
    while start < len(a_str):
        start = a_str.find(sub, start)

        if start == -1:
            return res

        res.append(start)
        start = start + len(sub)  # use start += 1 to find overlapping matches
    return res


def get_between_chars(str_, char_: str = '"'):
    try:
        indices = find_all(str_, char_)

        if len(indices) % 2 != 0:
            str_ = str_[indices[0] + len(char_) :]

        else:
            str_ = str_[(indices[0] + len(char_)) : indices[-1]]
    except:
        return None
    return str_

In [273]:
path_pattern: str = ".../data/llms_out/new/*.jsonl
glob(path_pattern)
# [x.split("\\")[-1].split(".")[0] for x  in glob(path_pattern)]

[]

In [264]:
from tqdm import tqdm
import re
from glob import glob


def _fetch_all_experiments(path_pattern: str = f"../data/llms_out/new/*.jsonl"):
    return [x.split("\\")[-1].split(".")[0] for x  in glob(path_pattern)]

def process_llm_generated_args(path):
    # 'text',  'original_text',
    text_keys = [
        "base",
        "content",
        "content_style",
        "content_style_ideology",
        "style",
        "style_ideology",
        "ideology",
    ]
    
    all_experiments = _fetch_all_experiments()
    df = pd.read_json(
        path_or_buf=path, lines=True
    )
    df = df.drop_duplicates(subset=["idx"], keep="last")
    #df = df.drop("original_text", axis=1)

    experiment = path.split("\\")[-1].split(".")[0]
    out = f"../data/llms_out/new/processed/{experiment}_processed.csv"

    print(experiment)
    model_type = experiment.split("_")[1]
    ideology = experiment.split("_")[0]

    exceptions_dict = {k:[] for k in all_experiments}
    
    exceptions_dict["liberal_chatgpt_0shot"] = [44708, 35204, 3176, 42440, 34248, 51598, 45614, 38169, 16602, 3259, 252,]
    exceptions_dict["conservative_chatgpt_0shot"] = [79903, 17012, 66698, 824, 43599, 46682, 78779, 78643, 58514, 43596, 25775, 32238, 61381]
    exceptions_dict["liberal_chatgpt_1shot"] = [252, 20310, 18397, 9161, 45614]
    exceptions_dict["conservative_chatgpt_1shot"] = [79903, 66698, 40673, 23110, 13417, 46682, 78778, 78643, 11374, 43596, 9181, 49307, 48861,] # 10968 "Vote for me for bacon and strippers"
  
    dismiss_dict = {k:{} for k in all_experiments}
    dismiss_dict["conservative_llamav2_0shot"] = {
        2323:["style"],
        10968:["content"],
        10740 : ["ideology", "style_ideology", "content_style_ideology", "content_style", "content", "base"],
        24053 :["content_style"],
        36097: ["base", "ideology"],
        77305: ["content_style_ideology"],
        80076: ["style_ideology"],
        52539: ["content_style_ideology"],
        64942: ["content_style_ideology"]
        }
    
    arr = []
    debug_idx = []
    correct = 7 * len(df)
    processing_logs = []

    phrase_delimiters = [f"Effective Argument for Readers with {ideology} Ideology:".lower(),
                        f"paraphrased Argument for Readers with {ideology} Ideology:".lower(),
                        "Effective argument:".lower(),
                        "paraphrased argument:".lower(),
                        "Here is a paraphrased version of the ineffective argument:".lower(),
                        "Here is a possible paraphrased argument that maintains the original length and stance of the original argument:".lower(),
                        "Here is my paraphrased version of the argument:".lower(),
                        "Here is my paraphrased version:".lower(),
                        "Here is a paraphrased version of the argument:".lower()]

    no_response = ["I cannot comply with your request",
                "I cannot fulfill your request",
                "I cannot provide a paraphrased argument"]
    format_type = ""
    for idx, row in tqdm(df.iterrows()):

        for col in text_keys:
            exception =""
            refuse_to_respond = False
            ## TO DISMISS
            if idx in dismiss_dict[experiment].keys() and col in dismiss_dict[experiment][idx]:
                success = False
                processing_logs.append({
                    "idx": row['idx'],
                    "prompt": col,
                    "ineffective_argument": row["text"],
                    "generated": row[col],
                    "effective_argument": "",
                    "cannot_paraphrase": row[col].find("I cannot provide a paraphrased argument") > -1,
                    "type": "DISMISSED", ## The argument is between " " or backticks
                    "flag": False,
                    "success": success,
                    "incontent_ideology_mentioned": cleaned_arg.lower().find(f"{ideology}") > -1 and row["original_text"].lower().find(f"{ideology}") <0 ,
                    "ideology_mentioned": row[col].lower().find(f"{ideology}") > -1 and row["original_text"].lower().find(f"{ideology}") <0 ,
                    "exception": "Exceptionally dismissed",
                    "llm_out_msge":""
                    })
                continue
            success = False
            flag = False
            delimiter_num =0
            
            ## Phrase delimiter
            format_type = "Not between delimiters and Not rephrased"
            phrase_delimiters_dic = {k:row[col].lower().find(k) for k in phrase_delimiters}
            latest_phrase_delim = max(phrase_delimiters_dic.items(), key=lambda x: x[1])[0]
            paraphras_idx = phrase_delimiters_dic[latest_phrase_delim]  
            if paraphras_idx > -1:
                sub = row[col][paraphras_idx+len(latest_phrase_delim):].strip()
                success = True
                format_type = "between a phrase delimiter"
                delimiter_num = len(latest_phrase_delim)
                
                
            ## Double quotes
            if not success:
                delimiter_num = 2
                sub = get_between_chars(row[col]) 
                format_type = "between quotations" if sub is not None else format_type
                success = sub is not None
            
            ## Sticks
            if not success:
                # if model_type == "llamav2":
                sub = get_between_chars(row[col].replace("´´´´", "```"), char_="```")
                delimiter_num = 6 if sub is not None else delimiter_num
                format_type = "between sticks" if sub is not None else format_type
                success = sub is not None

            if not success:
                
                # Search 
                format_type = "not between quotations/sticks/phrases"
                sub =row[col] 
                for nr in no_response:
                    if sub.lower().find(nr.lower()):
                        refuse_to_respond = True
                        format_type = "refused to respond"
                        success=False
                if not refuse_to_respond:
                    print(f"!!!!!!! Not by quotes, not sticks and not phrases")
                    print(sub)


            # Exceptional handling - some unmapped quotes 
            exceptions = exceptions_dict[experiment]
            if  row['idx'] in exceptions:
                success = True
                exception = "exceptionally combine all"
            cleaned_arg = row[col] if row['idx'] in exceptions else sub
            if row['idx'] == 18397 and experiment == "liberal_chatgpt_1shot" and col == "ideology":
                cleaned_arg = sub
                success = True
                exception = "exceptionally combine all"
            
            if len(cleaned_arg) < (len(row[col]) - delimiter_num):
                i = row[col].find(cleaned_arg) 
                # success = True
                flag_ = True
                llm_out_msge = f"{row[col][0:i]}...{row[col][len(cleaned_arg)+i: ]}"
                #print(
                #    f"######### {row['idx']} generated text for {col}"
                #    # f"{sub} -\n ****kept:*** \n {row[col][i: len(sub)+1]}\n"
                #    f"\n ****DISMISSED:*** \n {row[col][0:i]}...{row[col][len(cleaned_arg)+i: ]}"
                #    "\n---------------------------------------------------------------------------------\n"
                #)
                debug_idx.append(row["idx"])
                #paraphrase_found = cleaned_arg.lower().find("paraphrased argument:")
                #if  paraphrase_found > -1:
                #    cleaned_arg = cleaned_arg[(paraphrase_found+len("paraphrased argument:")):]


                correct = correct - (0 if success else 1)
            else:
                flag_ = False
                exception = "generated equal effective" if not refuse_to_respond else ""
            i = row[col].find(cleaned_arg) 
            processing_logs.append({
                        "idx": row['idx'],
                        "prompt": col,
                        "ineffective_argument": row["text"],
                        "generated": row[col],
                        "effective_argument": cleaned_arg.replace('´´´´', '').replace("'''", "").replace("```", "").strip() if success else "",
                        "cannot_paraphrase": row[col].find("I cannot provide a paraphrased argument") > -1,
                        "type": format_type, ## The argument is between " " or backticks
                        "flag": flag_,
                        "success": success,
                        "incontent_ideology_mentioned": cleaned_arg.lower().find(f"{ideology}") > -1 and row["original_text"].lower().find(f"{ideology}") <0 ,
                        "ideology_mentioned": row[col].lower().find(f"{ideology}") > -1 and row["original_text"].lower().find(f"{ideology}") <0 ,
                        "exception": exception,
                        "llm_out_msge": f"{row[col][0:i]}-effective_argument-{row[col][len(cleaned_arg)+i: ]}" if i>-1 and success else ""
                    })
    processed_df = pd.DataFrame(processing_logs)
    processed_df.to_csv(out)
    return processed_df
        # print(f"result: {result}")
        # print(isinstance(result, dict))



In [265]:
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter

def get_n_grams(col, n=3):
    # Tokenize the text data
    text_data = col
    tokens = []
    for sentence in text_data:
        sentence = re.sub('[^a-zA-Z0-9]+', ' ', sentence).lower()
        sentence = sentence.replace("and", "")
        sentence = sentence.replace("that", "")
        sentence = sentence.replace("to", "")

        tokens += word_tokenize(sentence)

    # Generate all possible 3-grams
    n_grams = []
    for i in range(len(tokens)-(n-1)):
        n_grams.append(tuple(tokens[i:i+n]))

    # Count the frequency of each 3-gram
    frequencies = Counter(n_grams)

    # Sort the 3-grams by frequency
    top_n_grams = sorted(frequencies.items(), key=lambda x: x[1])[-30:]

    # Print or store the results
    print(top_n_grams)


#get_n_grams(error_logs_df[error_logs_df["success"] == False ]["llm_out_msge"], n=30)


In [270]:
from glob import glob
experiments = glob("../data/llms_out/new/*.jsonl")
experiments 

['../data/llms_out/new\\conservative_chatgpt_0shot.jsonl',
 '../data/llms_out/new\\conservative_chatgpt_1shot.jsonl',
 '../data/llms_out/new\\conservative_llamav2_0shot.jsonl',
 '../data/llms_out/new\\conservative_llamav2_0shot_steered_meanl0.2.jsonl',
 '../data/llms_out/new\\conservative_llamav2_0shot_steered_meanl0.5.jsonl',
 '../data/llms_out/new\\conservative_llamav2_0shot_steered_medianl0.2.jsonl',
 '../data/llms_out/new\\conservative_llamav2_1shot.jsonl',
 '../data/llms_out/new\\liberal_chatgpt_0shot.jsonl',
 '../data/llms_out/new\\liberal_chatgpt_1shot.jsonl',
 '../data/llms_out/new\\liberal_llamav2_0shot.jsonl',
 '../data/llms_out/new\\liberal_llamav2_0shot_steered_meanl0.2.jsonl',
 '../data/llms_out/new\\liberal_llamav2_0shot_steered_meanl0.5.jsonl',
 '../data/llms_out/new\\liberal_llamav2_0shot_steered_medianl0.2.jsonl',
 '../data/llms_out/new\\liberal_llamav2_13b_0shot.jsonl',
 '../data/llms_out/new\\liberal_llamav2_1shot.jsonl']

In [266]:

for experiment in experiments:
    print(f"\n*********************************** {experiment} ***********************************")
    processed = process_llm_generated_args(experiment)
    print(pd.crosstab(processed["prompt"], processed["type"]))

    print(pd.crosstab(processed["prompt"], processed["success"]))


*********************************** ../data/llms_out/new\conservative_chatgpt_0shot.jsonl ***********************************
conservative_chatgpt_0shot


0it [00:00, ?it/s]


KeyError: 'conservative_chatgpt_0shot'

In [263]:
with open('content1.txt', 'w', encoding="utf-8") as f:
    error_logs_df[error_logs_df["success"] == True][["content", "type"]].to_csv(f, header =False)

KeyError: "['content'] not in index"

In [238]:
import pandas as pd

error_logs_df = pd.DataFrame(processing_logs)
error_logs_df["type"].value_counts()

pd.crosstab(error_logs_df["prompt"], error_logs_df["type"])

type,between a phrase delimiter,between quotations,between sticks,refused to respond
prompt,,,,
base,415,39,3,72
content,433,28,3,65
content_style,450,26,3,50
content_style_ideology,237,105,8,179
ideology,214,107,11,197
style,436,26,4,63
style_ideology,220,97,3,209


In [239]:
pd.crosstab(error_logs_df["prompt"], error_logs_df["ideology_mentioned"])

ideology_mentioned,False,True
prompt,,
base,529,0
content,529,0
content_style,529,0
content_style_ideology,310,219
ideology,343,186
style,529,0
style_ideology,328,201


In [244]:
error_logs_df["llm_out_msge"].value_counts()

  Paraphrased argument:\n-effective_argument-                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  1250
  Paraphrased argument: -effective_argument-                                                                                                                                                                                                                                                        

In [ ]:

error_logs_df["success"].value_counts()


error_logs_df[error_logs_df["cannot_paraphrase"] == True]["prompt"].value_counts()



refused_df = error_logs_df[error_logs_df["cannot_paraphrase"] == True]
refused_df["success"].value_counts()


with open('content.txt', 'w') as f:
    error_logs_df[error_logs_df["flag"] == True]["content1"].to_csv(f, header=False)


In [ ]:
df.columns.tolist()

In [ ]:
10968 
df[df["idx"] == 10968]["content"].values

In [ ]:
df[df["idx"].isin(debug_idx)].iloc[0]["idx"]

In [ ]:
df = df.drop_duplicates(subset=["idx"], keep="last")
#df[df["len_orig"] < 4096]

len(df)

In [ ]:
[x for x in df.columns.tolist() if not x.startswith("len_")]